# Training classifiers on HOG Data
#### For extraction of features, see extractFeatures.ipynb

conda env- make sure to switch to python3 kernel, faus_dl

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import sklearn
from sklearn import datasets, svm, metrics
import bcolz
import os, sys
import datetime as dt

In [2]:
import pickle

In [3]:
DISFA_path='/media/amogh/Stuff/CMU/datasets/DISFA_data/'
DISFA_AU_path=DISFA_path+'ActionUnit_Labels/'
print(DISFA_AU_path)
Videos_right_path=DISFA_path+'Videos_RightCamera/'
print(Videos_right_path)

/media/amogh/Stuff/CMU/datasets/DISFA_data/ActionUnit_Labels/
/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/


In [4]:
import mnist_helpers

Helper Functions

## FAU4_1

Loading the saved features:

In [14]:
dir_features_hog_1_fau4_1=DISFA_path+"/features/hog_1/fau4_1/"
dir_features_hog_1_fau4_1

'/media/amogh/Stuff/CMU/datasets/DISFA_data//features/hog_1/fau4_1/'

In [15]:
features_fau4_1_hog_1_positives=bcolz.carray(rootdir=dir_features_hog_1_fau4_1+'positives/',mode='r')
print("features_fau4_1_hog_1_positives loaded, shape is: ",features_fau4_1_hog_1_positives.shape)

features_fau4_1_hog_1_positives loaded, shape is:  (2000, 448)


In [16]:
features_fau4_1_hog_1_negatives=bcolz.carray(rootdir=dir_features_hog_1_fau4_1+'negatives/',mode='r')
print("features_fau4_1_hog_1_negatives loaded, shape is : ",features_fau4_1_hog_1_negatives.shape)

features_fau4_1_hog_1_negatives loaded, shape is :  (1898, 448)


Loading test_features:

In [17]:
features_fau4_1_hog_1_positives_test=bcolz.carray(rootdir=dir_features_hog_1_fau4_1+'positives_test/',mode='r')
print("features_fau4_1_hog_1_positives_test loaded, shape is: ",features_fau4_1_hog_1_positives_test.shape)

features_fau4_1_hog_1_positives_test loaded, shape is:  (464, 448)


In [18]:
features_fau4_1_hog_1_negatives_test=bcolz.carray(rootdir=dir_features_hog_1_fau4_1+'negatives_test/',mode='r')
print("features_fau4_1_hog_1_negatives_test loaded, shape is : ",features_fau4_1_hog_1_negatives_test.shape)

features_fau4_1_hog_1_negatives_test loaded, shape is :  (499, 448)


Defining the training and testing data

In [22]:
train_no=1800
X=np.concatenate([features_fau4_1_hog_1_positives[:train_no],features_fau4_1_hog_1_negatives[:train_no]])
X.shape

(3600, 448)

In [24]:
Y=np.concatenate([np.repeat(1,train_no),np.repeat(0,train_no)])
Y.shape

(3600,)

In [25]:
X_train,X_test,y_train,y_test=sklearn.model_selection.train_test_split(X,Y,test_size=0.15,random_state=42)

Defining the parameter range for SVM:

In [26]:
gamma_range=np.outer(np.logspace(-3,0,4),np.array([1,5]))
gamma_range=gamma_range.flatten()
C_range=np.outer(np.logspace(-1,1,3),np.array([1,5]))
C_range=C_range.flatten()
parameters={'kernel':['rbf'],'C':C_range,'gamma':gamma_range}

In [27]:
svm_clsf=svm.SVC()
grid_clsf=sklearn.model_selection.GridSearchCV(estimator=svm_clsf,param_grid=parameters,n_jobs=1,verbose=2)

Start training and parameter search 

In [28]:
start_time=dt.datetime.now()
print('Start param searching at {}'.format(str(start_time)))
grid_clsf.fit(X_train,y_train)
elapsed_time=dt.datetime.now()-start_time
print('Elapsed time, param searching {}'.format(str(elapsed_time)))
sorted(grid_clsf.cv_results_.keys())

Start param searching at 2018-05-30 12:57:45.482446
Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ................... C=0.1, gamma=0.001, kernel=rbf, total=   4.3s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.9s remaining:    0.0s


[CV] ................... C=0.1, gamma=0.001, kernel=rbf, total=   4.1s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ................... C=0.1, gamma=0.001, kernel=rbf, total=   4.2s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................... C=0.1, gamma=0.005, kernel=rbf, total=   4.1s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................... C=0.1, gamma=0.005, kernel=rbf, total=   4.4s
[CV] C=0.1, gamma=0.005, kernel=rbf ..................................
[CV] ................... C=0.1, gamma=0.005, kernel=rbf, total=   4.2s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=0.01, kernel=rbf, total=   4.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=0.1, gamma=0.01, kernel=rbf, total=   4.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .

[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed: 11.5min finished


Elapsed time, param searching 0:11:35.283894


['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'mean_train_score',
 'param_C',
 'param_gamma',
 'param_kernel',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split0_train_score',
 'split1_test_score',
 'split1_train_score',
 'split2_test_score',
 'split2_train_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score',
 'std_train_score']

In [29]:
classifier=grid_clsf.best_estimator_
params=grid_clsf.best_params_

scores=grid_clsf.cv_results_['mean_test_score'].reshape(len(C_range),len(gamma_range))
# plot_param_space_scores(scores,C_range, gamma_range)

In [30]:
scores

array([[0.50915033, 0.60130719, 0.78398693, 0.81143791, 0.83529412,
        0.83267974, 0.8003268 , 0.50915033],
       [0.60163399, 0.75980392, 0.81699346, 0.85784314, 0.88660131,
        0.92614379, 0.91862745, 0.50915033],
       [0.78039216, 0.81601307, 0.84019608, 0.89509804, 0.92287582,
        0.94411765, 0.93888889, 0.61437908],
       [0.81699346, 0.86699346, 0.89509804, 0.94248366, 0.95326797,
        0.95490196, 0.95163399, 0.64379085],
       [0.84215686, 0.89542484, 0.91699346, 0.9503268 , 0.95196078,
        0.95490196, 0.95163399, 0.64379085],
       [0.89705882, 0.92745098, 0.93529412, 0.94542484, 0.94803922,
        0.95490196, 0.95163399, 0.64379085]])

In [31]:
expected = y_test
predicted = classifier.predict(X_test)
print(predicted)
print("predicted and expected are same in: ", np.sum(expected==predicted)," out of ", expected.shape[0])

print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted)))
      
cm = metrics.confusion_matrix(expected, predicted)
print("Confusion matrix:\n%s" % cm)

# plot_confusion_matrix(cm)

print("Accuracy={}".format(metrics.accuracy_score(expected, predicted)))


[1 1 0 0 0 1 1 1 0 1 0 1 1 1 1 0 0 1 1 0 1 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1
 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 1 0
 0 1 1 0 1 1 1 1 0 0 1 0 0 0 0 1 1 1 1 0 0 1 0 1 1 1 0 1 1 0 1 1 0 0 1 1 1
 0 1 0 0 1 0 1 0 1 0 0 1 1 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 0 1 1 0 0 0 1 0 1
 1 1 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 1 0 1
 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 1 1 0 1 1 1 1 0
 0 0 0 0 0 1 1 0 0 0 1 1 1 1 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1
 1 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 0 0 0 0 1 0 0 0
 1 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 0 1 1 1 1 0 1 0
 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 0 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 0 0 0
 0 0 0 1 0 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 1 1 1 0 1 0 0
 1 1 1 0 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1
 0 1 1 1 1 0 0 1 1 1 0 0 0 0 0 1 1 1 0 0 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 1 0 0 0 0 1 0 

In [40]:
print(classifier.predict(features_fau4_1_hog_1_positives_test).sum()," out of ", len(features_fau4_1_hog_1_positives_test))

14  out of  464


In [41]:
print(classifier.predict(features_fau4_1_hog_1_negatives_test).sum()," out of ", len(features_fau4_1_hog_1_positives_test))

0  out of  464


**Saving the model to disk**

In [32]:
DISFA_path

'/media/amogh/Stuff/CMU/datasets/DISFA_data/'

In [33]:
models_path=DISFA_path+'models/'
if not os.path.exists(models_path):
    os.makedirs(models_path)

In [34]:
with open(models_path+'fau4_1_hog_1.pickle','wb') as handle:
    pickle.dump(classifier,handle)

**Loading a model from disk**

In [35]:
with open(models_path+'fau4_1_hog_1.pickle','rb') as handle:
    clf1=pickle.load(handle)

**Calculating features for a cropped image from disk**

In [18]:
def calcHOGfeatures(cropped_im_path):
    #instead of 448, put the dimension of the HOG vector
    features_fau4_1_hog_1_negatives=np.empty((0,448))
    im=cv2.imread(cropped_im_path,0)
    fd, hog_image = hog(im, orientations=8, pixels_per_cell=(8,8),cells_per_block=(1, 1), visualise=True)
    print ("The shape of HOG features vector is: ",features_fau4_1_hog_1_negatives.shape)
    return fd

In [23]:
dir_cropped_FAU4_1=DISFA_path+"/features/cropped_images/FAU4_1/"
print(dir_cropped_FAU4_1)
dir_cropped_FAU4_1_positives=dir_cropped_FAU4_1+"positives/"
print(dir_cropped_FAU4_1_positives)
dir_cropped_FAU4_1_negatives=dir_cropped_FAU4_1+"negatives/"
print(dir_cropped_FAU4_1_negatives)

/media/amogh/Stuff/CMU/datasets/DISFA_data//features/cropped_images/FAU4_1/
/media/amogh/Stuff/CMU/datasets/DISFA_data//features/cropped_images/FAU4_1/positives/
/media/amogh/Stuff/CMU/datasets/DISFA_data//features/cropped_images/FAU4_1/negatives/


In [24]:
list_cropped_fau4_1_positive_files=os.listdir(dir_cropped_FAU4_1_positives)
list_cropped_fau4_1_positive_files[0]

'RightVideoSN001_1340.png'

In [44]:
classifier.predict(features_fau4_1_hog_1_positives)

array([1, 1, 1, ..., 1, 1, 1])

#### Automating loading features, training process 

In [5]:
dir_features_hog_2_fau4_1=DISFA_path+"/features/hog_2/fau4_1/"
dir_features_hog_2_fau4_1

'/media/amogh/Stuff/CMU/datasets/DISFA_data//features/hog_2/fau4_1/'

In [14]:
def loadTrainData(features_folder):
    #loading features
    features_positives=bcolz.carray(rootdir=features_folder+'positives/',mode='r')
    print("features_positives loaded, shape is: ",features_positives.shape)
    features_negatives=bcolz.carray(rootdir=features_folder+'negatives/',mode='r')
    print("features_negatives loaded, shape is : ",features_negatives.shape)
    
#     features_positives_test=bcolz.carray(rootdir=features_folder+'positives_test/',mode='r')
#     print("features_positives_test loaded, shape is: ",features_positives_test.shape)
#     features_negatives_test=bcolz.carray(rootdir=features_folder+'negatives_test/',mode='r')
#     print("features_negatives_test loaded, shape is: ",features_negatives_test.shape)
    
    #setting up the training data
    train_no=np.min([features_positives.shape[0],features_negatives.shape[0]])
    X=np.concatenate([features_positives[:train_no],features_negatives[:train_no]])
    print("training data shape is: ",X.shape)
    #setting labels/targets
    Y=np.concatenate([np.repeat(1,train_no),np.repeat(0,train_no)])
    print("labels shape is: ",Y.shape)
    return X, Y

In [15]:
def trainModel(X_train,y_train):
    #setup
    gamma_range=np.outer(np.logspace(-3,0,4),np.array([1,5]))
    gamma_range=gamma_range.flatten()
    C_range=np.outer(np.logspace(-1,1,3),np.array([1,5]))
    C_range=C_range.flatten()
    parameters={'kernel':['rbf'],'C':C_range,'gamma':gamma_range}
    svm_clsf=svm.SVC()
    grid_clsf=sklearn.model_selection.GridSearchCV(estimator=svm_clsf,param_grid=parameters,n_jobs=1,verbose=2)
    #train
    start_time=dt.datetime.now()
    print('Start param searching at {}'.format(str(start_time)))
    grid_clsf.fit(X_train,y_train)
    elapsed_time=dt.datetime.now()-start_time
    print('Elapsed time, param searching {}'.format(str(elapsed_time)))
    sorted(grid_clsf.cv_results_.keys())
    #choose classifier
    classifier=grid_clsf.best_estimator_
    params=grid_clsf.best_params_
    scores=grid_clsf.cv_results_['mean_test_score'].reshape(len(C_range),len(gamma_range))
    print("scores are: ", scores)
    return classifier, params,scores

In [ ]:
X,Y=loadTrainData(dir_features_hog_2_fau4_1)
trainModel(X,Y)

features_positives loaded, shape is:  (1999, 1056)
features_negatives loaded, shape is :  (2000, 1056)
training data shape is:  (3998, 1056)
labels shape is:  (3998,)
Start param searching at 2018-06-06 14:51:27.386642
Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
